In [ ]:
!pip install pandas
!pip install pandas sqlalchemy
!pip install torch
!pip install transformers

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, inspect
import sqlite3
from auto_gptq import AutoGPTQForCausalLM
from transformers import (
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    pipeline
)
import torch

In [ ]:
# Path to your CSV file
csv_file_path = 'take_home_dataset.csv' #Change it as per your file path

# Create an SQLite engine in memory (or you can create it in a file)
engine = create_engine('sqlite:///:memory:')

# Read the CSV file using pandas
df = pd.read_csv(csv_file_path, delimiter=';')

# Write the data frame to a SQL table named 'my_table'
df.to_sql('shipments', con=engine, index=False, if_exists='replace')

# Use the inspect function from sqlalchemy to describe the table schema
inspector = inspect(engine)
columns = inspector.get_columns('shipments')

print("Table Schema:")
for column in columns:
    print(f"Name: {column['name']}, Type: {column['type']}")
    print()
#Use the schema provided to refer and accordingly write schema for our text2sql model

In [ ]:
model_name = 'shashwat1225/Text2sql-Llama2-13b' #OpenSource model finetuned for text2sql

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Ensure the pipeline uses the specified device
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, device=0 if device == 'cuda' else -1)


In [ ]:
table = "CREATE TABLE shipments ( Order_date DATE, Product_Category ENUM('Apparel', 'Cosmetics & Personal Care', 'Groceries', 'Toys & Games', 'Electronics'), Delivery_distance INT, Mode_Of_Transport ENUM('Less Than Truckload', 'Full Truckload', 'Air Freight', 'Intermodal & Rail Shipments'));"

#Using four columns out of 19 and declaring the schema for the model to interpret
#Schema with indentation for better readability
"""
CREATE TABLE shipments ( 
    Order_date DATE, 
    Product_Category ENUM(
        'Apparel', 
        'Cosmetics & Personal Care', 
        'Groceries', 
        'Toys & Games', 
        'Electronics'), 
    Delivery_distance INT, 
    Mode_Of_Transport ENUM(
        'Less Than Truckload', 
        'Full Truckload', 
        'Air Freight', 
        'Intermodal & Rail Shipments')
);
"""

In [ ]:
question = input() #Enter the question you want to ask
#Example question: "Find the apparel product with delivery distances greater than 400 along with air transport"

In [ ]:
prompt = f"""[INST] Write SQLite query to answer the following question given the database schema. Please wrap your code answer using ```: Schema: {table} Question: {question} [/INST] Here is the SQLite query to answer to the question: {question}: [/INST] Stick to the question, don't assume in any other condition that is not provided in the question:``` """

In [ ]:
ans = pipe(prompt, max_new_tokens=100)
#print(ans[0]['generated_text'])
sql_query = ans[0]['generated_text'].split('```')[2].split('\n')[1].strip()
print(sql_query) #Prints the SQL query generated by the model

In [ ]:
# Running the query and returning the results as a DataFrame
result_df = pd.read_sql_query(sql_query, con=engine)

In [ ]:
# Display the result
print(result_df)